# **Office choice**

In [4]:
import pandas as pd
from pymongo import MongoClient
import folium
from shapely.geometry import Polygon

In [5]:
shops = pd.read_csv("../output/shops.csv")
display(shops.head(),shops.shape)

,Unnamed: 0,category,name,location,latitude,longitude
0,0,airport,San Francisco International Airport,"{'type': 'Point', 'coordinates': [-122.3789554...",37.621313,-122.378955
1,0,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.4042684...",37.792814,-122.404268
2,1,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.38783, ...",37.741198,-122.387830
3,2,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.40385, ...",37.784460,-122.403850
4,3,Starbucks,Starbucks,"{'type': 'Point', 'coordinates': [-122.406831,...",37.789348,-122.406831


(107, 6)

In [6]:
shops.category.value_counts()

Starbucks     60
pub           40
vegan rest     6
airport        1
Name: category, dtype: int64

### **Localization of shops on the map**

In [7]:
# coord SF [37.774929,-122.4194183]
m = folium.Map(
    location=[37.774929,-122.4194183],
    zoom_start=13.5,
    tiles='OpenStreetMap'
)
for i in range(len(shops.index)):
    dict_color ={"airport":"black","Starbucks":"blue","vegan rest":"green","pub":"red"}
    folium.Marker(
    location=[float(shops.iloc[i][4]),float(shops.iloc[i][5])],
    popup=f'{shops.iloc[i][1]}\n[{shops.iloc[i][4]},{shops.iloc[i][5]}]',
    icon=folium.Icon(color=dict_color[shops.iloc[i][1]])).add_to(m)
    
aline=folium.PolyLine(locations=([37.7683125,-122.42168],[37.766884,-122.409623]),weight=5,color = 'black') 
m.add_child(aline) 
aline=folium.PolyLine(locations=([37.766884,-122.409623],[37.7628754,-122.4104225]),weight=5,color = 'black') 
m.add_child(aline) 
aline=folium.PolyLine(locations=([37.7628754,-122.4104225],[37.764626,-122.4221595]),weight=5,color = 'black') 
m.add_child(aline) 
aline=folium.PolyLine(locations=([37.764626,-122.4221595],[37.7683125,-122.42168]),weight=5,color = 'black') 
m.add_child(aline)

display(m)

Inside the drawn polygon are located the three different types of required shops within a reduced radius.

#### **Choosing a office inside the polygon**

In [8]:
client = MongoClient("mongodb://localhost/datamad0320")
db = client.get_database()

In [9]:
cur = db.companiesSF.find({"location":{
    "$geoWithin": {"$geometry": {
             "type" : "Polygon" ,"coordinates":[[[-122.42168,37.7683125],[-122.409623,37.766884],
                                                [-122.4104225,37.7628754],[-122.4221595,37.764626],
                                               [-122.42168,37.7683125]]]}}}})

In [10]:
offices_near = pd.DataFrame(cur)

In [11]:
offices_near

,_id,name,category_code,number_of_employees,founded_year,location,latitude,longitude,total_money_raised ($k)
0,5e9dd90f55122cdd5bda18c7,CrowdFlower,enterprise,49.0,2007.0,"{'type': 'Point', 'coordinates': [-122.4204782...",37.765276,-122.420478,13200.0
1,5e9dd90f55122cdd5bda1761,TuneUp,music,NaN,2007.0,"{'type': 'Point', 'coordinates': [-122.4119381...",37.765420,-122.411938,15600.0
2,5e9dd90f55122cdd5bda1868,Vinfolio,ecommerce,68.0,2003.0,"{'type': 'Point', 'coordinates': [-122.410752,...",37.763293,-122.410752,10600.0


In [12]:
m = folium.Map(
    location=[37.765,-122.419],
    zoom_start=15,
    tiles='OpenStreetMap'
)
for i in range(len(shops.index)):
    dict_color ={"airport":"black","Starbucks":"blue","vegan rest":"green","pub":"red"}
    folium.Marker(
    location=[float(shops.iloc[i][4]),float(shops.iloc[i][5])],
    popup=f'{shops.iloc[i][1]}\n[{shops.iloc[i][4]},{shops.iloc[i][5]}]',
    icon=folium.Icon(color=dict_color[shops.iloc[i][1]])).add_to(m)
    
for i in range(len(offices_near.index)):
    folium.Marker(
    location=[float(offices_near.iloc[i][6]),float(offices_near.iloc[i][7])],
    popup=offices_near.iloc[i][1],
    icon=folium.Icon(color="black")).add_to(m)

aline=folium.PolyLine(locations=([37.7683125,-122.42168],[37.766884,-122.409623]),weight=5,color = 'black') 
m.add_child(aline) 
aline=folium.PolyLine(locations=([37.766884,-122.409623],[37.7628754,-122.4104225]),weight=5,color = 'black') 
m.add_child(aline) 
aline=folium.PolyLine(locations=([37.7628754,-122.4104225],[37.764626,-122.4221595]),weight=5,color = 'black') 
m.add_child(aline) 
aline=folium.PolyLine(locations=([37.764626,-122.4221595],[37.7683125,-122.42168]),weight=5,color = 'black') 
m.add_child(aline)

display(m)

To choose the best located company inside the polygon, the center of this polygon is calculated

#### **Choosing the office closest to the center of the polygon**

In [13]:
# Calculate the center of the polygon

coords =[(-122.42168,37.7683125),(-122.409623,37.766884),(-122.4104225,37.7628754),(-122.4221595,37.764626)]
poly = Polygon(coords)
print(poly.centroid)

POINT (-122.4158781108198 37.76566921205165)


In [14]:
location = db.companiesSF.find({"$and":[{"location":{
    "$geoWithin": {"$geometry": {
             "type" : "Polygon" ,"coordinates":[[[-122.42168,37.7683125],[-122.409623,37.766884],
                                                [-122.4104225,37.7628754],[-122.4221595,37.764626],
                                               [-122.42168,37.7683125]]]}}}}, {"location": {
         "$near": {
           "$geometry": {"type" : "Point" ,"coordinates":[-122.4158781108198,37.76566921205165]},
           "$maxDistance": 600}}}]},{"name":1,"_id":0,"latitude":1,"longitude":1})

In [15]:
pd.DataFrame(location)

,name,latitude,longitude
0,TuneUp,37.765420,-122.411938
1,CrowdFlower,37.765276,-122.420478
2,Vinfolio,37.763293,-122.410752


#### **Conclusion:** The ranking order of the companies by proximity to the center of the drawn polygon would be:
1. TuneUP      [37.765420, -122.411938]
2. CrowdFlower [37.765276, -122.420478]
3. Vinfolio    [37.763293, -122.410752]